In [16]:
import sys
import json
import pickle
import pyspark
from functools import reduce
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import udf, mean, lit, stddev, col, expr, when
from pyspark.sql.types import DoubleType, ArrayType, ShortType, LongType, IntegerType
import pandas as pd
from collections import OrderedDict
from datetime import date
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

from pyspark.ml import Pipeline, PipelineModel

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import DecisionTreeClassifier

from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorIndexer, IndexToString
from pyspark.ml import PipelineModel

In [2]:
#sc = SparkContext.getOrCreate()
sql = sqlContext #SQLContext.getOrCreate(sc)

In [3]:
STORAGE_ACCOUNT_SUFFIX = 'core.windows.net'
STORAGE_ACCOUNT_NAME = os.environ['STORAGE_ACCOUNT_NAME']

wasbUrlOutput = "wasb://{0}@{1}.blob.{2}/features.parquet".format(
            'intermediate',
            STORAGE_ACCOUNT_NAME,
            STORAGE_ACCOUNT_SUFFIX)

dfa = sql.read.parquet(wasbUrlOutput)
dfa.printSchema()

root
 |-- EnqueuedTimeUtc: string (nullable = true)
 |-- machineID: string (nullable = true)
 |-- ambient_temperature: double (nullable = true)
 |-- ambient_pressure: double (nullable = true)
 |-- speed: double (nullable = true)
 |-- temperature: double (nullable = true)
 |-- pressure: double (nullable = true)
 |-- f0: double (nullable = true)
 |-- f1: double (nullable = true)
 |-- f2: double (nullable = true)
 |-- a0: double (nullable = true)
 |-- a1: double (nullable = true)
 |-- a2: double (nullable = true)
 |-- temperature_n: double (nullable = true)
 |-- pressure_n: double (nullable = true)
 |-- f0_n: double (nullable = true)
 |-- f1_n: double (nullable = true)
 |-- f2_n: double (nullable = true)
 |-- a0_n: double (nullable = true)
 |-- a1_n: double (nullable = true)
 |-- a2_n: double (nullable = true)



In [4]:
rdd = sc.parallelize([('MACHINE-008', 'F01'),('MACHINE-009', 'F02')])

failures_df = rdd.toDF(['machineID','failure'])
failures_df.printSchema()

root
 |-- machineID: string (nullable = true)
 |-- failure: string (nullable = true)



In [9]:
features = sorted([c for c in dfa.columns if c not in ['machineID', 'EnqueuedTimeUtc']])

# assemble features
va = VectorAssembler(inputCols=features, outputCol='features')

# this is a hack!! In the current simulated dataset, there are 4 machines: machine 0 is "good," 
# whereas the rest are experiencing different failures. This will eventually be produced by merging 
# telemetry with the "maintenance log"
feat_data = va.transform(dfa).join(failures_df, 'machineID', 'left_outer').fillna({'failure':'None'})

dfa.unpersist()
feat_data.persist(pyspark.StorageLevel.DISK_ONLY)
feat_data.limit(10).toPandas().head()

,machineID,EnqueuedTimeUtc,ambient_temperature,ambient_pressure,speed,temperature,pressure,f0,f1,f2,...,temperature_n,pressure_n,f0_n,f1_n,f2_n,a0_n,a1_n,a2_n,features,failure
0,MACHINE-009,2018-03-26T22:34:55.4790000Z,20.0,101.0,806.0,80.101178,1612.0,27.0,242.0,161.0,...,0.099381,2.0,0.033499,0.300248,0.199752,14.773351,8.566254,2.285134,"[11907.3210649, 14.7733511971, 6904.40076309, ...",F02
1,MACHINE-009,2018-03-26T22:34:56.5100000Z,20.0,101.0,806.0,80.105199,1612.0,27.0,242.0,161.0,...,0.099386,2.0,0.033499,0.300248,0.199752,14.769033,8.555965,2.291768,"[11903.8403024, 14.7690326332, 6896.1074441, 8...",F02
2,MACHINE-009,2018-03-26T22:34:57.5260000Z,20.0,101.0,806.0,80.109187,1612.0,27.0,242.0,26.0,...,0.099391,2.0,0.033499,0.300248,0.032258,14.741047,8.578353,2.311062,"[11881.2835531, 14.741046592, 6914.15226527, 8...",F02
3,MACHINE-009,2018-03-26T22:34:58.5100000Z,20.0,101.0,806.0,80.113143,1612.0,27.0,242.0,161.0,...,0.099396,2.0,0.033499,0.300248,0.199752,14.807012,8.549248,2.274928,"[11934.4519184, 14.8070123057, 6890.69350123, ...",F02
4,MACHINE-009,2018-03-26T22:34:59.4980000Z,20.0,101.0,806.0,80.117067,1612.0,27.0,242.0,161.0,...,0.099401,2.0,0.033499,0.300248,0.199752,14.785386,8.550428,2.324057,"[11917.0213785, 14.7853863256, 6891.64464942, ...",F02


In [33]:
# set maxCategories so features with > 10 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", 
                               outputCol="indexedFeatures", 
                               maxCategories=10).fit(feat_data)

# fit on whole dataset to include all labels in index
labelIndexer = StringIndexer(inputCol="failure", outputCol="indexedLabel").fit(feat_data)

deIndexer = IndexToString(inputCol = "prediction", outputCol = "predictedFailure", labels = labelIndexer.labels)

In [34]:
training, test = feat_data.randomSplit([0.8, 0.2], seed=12345)
print(training.count())
print(test.count())
feat_data.limit(5).toPandas().head()

34788
8593


,machineID,EnqueuedTimeUtc,ambient_temperature,ambient_pressure,speed,temperature,pressure,f0,f1,f2,...,temperature_n,pressure_n,f0_n,f1_n,f2_n,a0_n,a1_n,a2_n,features,failure
0,MACHINE-009,2018-03-26T22:34:55.4790000Z,20.0,101.0,806.0,80.101178,1612.0,27.0,242.0,161.0,...,0.099381,2.0,0.033499,0.300248,0.199752,14.773351,8.566254,2.285134,"[11907.3210649, 14.7733511971, 6904.40076309, ...",F02
1,MACHINE-009,2018-03-26T22:34:56.5100000Z,20.0,101.0,806.0,80.105199,1612.0,27.0,242.0,161.0,...,0.099386,2.0,0.033499,0.300248,0.199752,14.769033,8.555965,2.291768,"[11903.8403024, 14.7690326332, 6896.1074441, 8...",F02
2,MACHINE-009,2018-03-26T22:34:57.5260000Z,20.0,101.0,806.0,80.109187,1612.0,27.0,242.0,26.0,...,0.099391,2.0,0.033499,0.300248,0.032258,14.741047,8.578353,2.311062,"[11881.2835531, 14.741046592, 6914.15226527, 8...",F02
3,MACHINE-009,2018-03-26T22:34:58.5100000Z,20.0,101.0,806.0,80.113143,1612.0,27.0,242.0,161.0,...,0.099396,2.0,0.033499,0.300248,0.199752,14.807012,8.549248,2.274928,"[11934.4519184, 14.8070123057, 6890.69350123, ...",F02
4,MACHINE-009,2018-03-26T22:34:59.4980000Z,20.0,101.0,806.0,80.117067,1612.0,27.0,242.0,161.0,...,0.099401,2.0,0.033499,0.300248,0.199752,14.785386,8.550428,2.324057,"[11917.0213785, 14.7853863256, 6891.64464942, ...",F02


In [35]:
model_type = 'RandomForest' # Use 'DecisionTree', or 'RandomForest'

# train a model.
if model_type == 'DecisionTree':
    model = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures",
                                      # Maximum depth of the tree. (>= 0) 
                                      # E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'
                                      maxDepth=15,
                                      # Max number of bins for discretizing continuous features. 
                                      # Must be >=2 and >= number of categories for any categorical feature.
                                      maxBins=32, 
                                      # Minimum number of instances each child must have after split. 
                                      # If a split causes the left or right child to have fewer than 
                                      # minInstancesPerNode, the split will be discarded as invalid. Should be >= 1.
                                      minInstancesPerNode=1, 
                                      # Minimum information gain for a split to be considered at a tree node.
                                      minInfoGain=0.0, 
                                      # Criterion used for information gain calculation (case-insensitive). 
                                      # Supported options: entropy, gini')
                                      impurity="gini")

    ##=======================================================================================================================
    #elif model_type == 'GBTClassifier':
    #    cls_mthd = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")
    ##=======================================================================================================================
else:    
    model = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", 
                                      # Passed to DecisionTreeClassifier
                                      maxDepth=15, 
                                      maxBins=32, 
                                      minInstancesPerNode=1, 
                                      minInfoGain=0.0,
                                      impurity="gini",
                                      # Number of trees to train (>= 1)
                                      numTrees=50, 
                                      # The number of features to consider for splits at each tree node. 
                                      # Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n].
                                      featureSubsetStrategy="sqrt", 
                                      # Fraction of the training data used for learning each 
                                      # decision tree, in range (0, 1].' 
                                      subsamplingRate = 0.632)

# chain indexers and model in a Pipeline
pipeline_cls_mthd = Pipeline(stages=[labelIndexer, featureIndexer, model, deIndexer])

# train model.  This also runs the indexers.
model_pipeline = pipeline_cls_mthd.fit(training)

In [52]:
# make predictions. The Pipeline does all the same operations on the test data
predictions = model_pipeline.transform(test)
# predictions.limit(5).toPandas().head()
# Create the confusion matrix for the multiclass prediction results
# This result assumes a decision boundary of p = 0.5
conf_table = predictions.stat.crosstab('failure', 'predictedFailure')
confuse = conf_table.toPandas().sort_values(by=['failure_predictedFailure'])
confuse.head()

,failure_predictedFailure,F01,F02,None
0,F01,847,0,0
2,F02,0,876,0
1,None,1,0,6869


In [69]:
model_pipeline.write().overwrite().save('../model')

import tarfile

tar = tarfile.open("../model.tar.gz", "w:gz")
tar.add("../model", arcname="model")
tar.close()